In [1]:
# True : 문서의 임베딩을 피클 파일에서 읽음
# False : 임베딩을 새로함
is_use_embeddings_from_pickle = True

# 문서 임베딩의 피클 파일명
embeddings_pickle_filename = 'embeddings_text-embedding-3-large_dim3072.pkl'

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
 # 색인 대상 문서 및 평가 데이터 다운로드
 #!wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
 #!tar -xzvf data.tar.gz

## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [4]:
# Elasticsearch 8.8.0 다운로드 및 압축 해제
#!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
#!tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz

# daemon user로 구동을 하기 위해 소유자 변경
#!chown -R daemon:daemon elasticsearch-8.8.0/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
#!umount /sys/fs/cgroup
#!apt install cgroup-tools

# Nori 형태소 분석기 설치
#!./elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori


In [5]:
# Elasticsearch python 패키지 설치
#!pip install elasticsearch==8.15.1

# OpenAI Python 패키지 설치
#!pip install openai==1.7.2

# 임베딩 생성을 위한 벡터 인코더 설치
#!pip install sentence-transformers==2.2.2

## 검색엔진 준비 - Elasticsearch




In [6]:
import os
import json
from elasticsearch import Elasticsearch, helpers
import time

In [7]:

# 엘라스틱서치의 데몬 인스턴스 만들기
# from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(['../../elasticsearch-8.8.0/bin/elasticsearch'],
#                   stdout=PIPE, stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(1)  # as daemon
#                  )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
#time.sleep(60)

In [8]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon    202043       1  2 02:24 ?        00:02:11 /data/ephemeral/home/elasticsearch-8.15.2/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate --enable-native-access=org.elasticsearch.nativeaccess,org.apache.lucene.core -XX:ReplayDataFile=logs/replay_pid%p.log -Djava.library.path=/data/ephemeral/home/elasticsearch-8.15.2/lib/platform/linux-x64:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib -Djna.library.path=/data/ephemeral/home/elasticsearch-8.15.2/lib/platform/linux-x64:/usr/java/packages/lib:/usr/lib

In [9]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
#!/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [10]:
es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = os.getenv('ES_PASSWORD')

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="../../elasticsearch-8.15.2/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-12254', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'k1o6AgbySE-7S71JVuwV0w', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [11]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [12]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
#model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")      # TODO 임베딩 잘 만드는 모델을 찾아보자.
#model = SentenceTransformer("jhgan/ko-sroberta-multitask")


# SetntenceTransformer를 이용하여 임베딩 생성
# def get_embedding(sentences):
#     return model.encode(sentences)

def get_embedding(text, model="text-embedding-3-large"):
   return client.embeddings.create(input=[text], model=model).data[0].embedding

def texts_to_tensor(texts, model="text-embedding-3-large"):
    embeddings = [get_embedding(text, model) for text in texts]
    return embeddings


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        
        #embeddings = get_embedding(contents)
        embeddings = texts_to_tensor(contents)
        
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")       ######## index 를 "test" 로 박아놨네??


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    #query_embedding = get_embedding([query_str])[0]
    query_embedding = texts_to_tensor([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embedding",
        #"query_vector": query_embedding.tolist(),
        "query_vector": query_embedding,
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)     ######## index 를 "test" 로 박아놨네??


/data/ephemeral/home/miniconda3/envs/colbert/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [13]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

In [14]:
import pickle

if not is_use_embeddings_from_pickle:
    embeddings = get_embeddings_in_batches(docs)
    
    # 나중을 위해 문서 임베딩을 피클 파일로 저장.
    with open(embeddings_pickle_filename, 'wb') as file:
        pickle.dump(embeddings, file)
else:
    # 피클 파일로부터 embeddings 리스트 복원
    with open(embeddings_pickle_filename, 'rb') as file:
        embeddings = pickle.load(file)

In [15]:
print(f'embedding dimension : {len(embeddings[0])}')
print(f'embeddings type : {type(embeddings)}')
print(f'embeddings length : {len(embeddings)}')
print(f'embeddings[0] : {embeddings[0]}')

embedding dimension : 3072
embeddings type : <class 'list'>
embeddings length : 4272
embeddings[0] : [0.03257926180958748, -0.008541478775441647, -0.00882707629352808, -0.01854267716407776, -0.011275810189545155, 0.015464572235941887, -0.016226164996623993, -0.019177338108420372, 0.011053678579628468, -0.07544003427028656, 0.036768026649951935, -0.008287614211440086, 0.014840489253401756, -0.00774815259501338, 0.0235670767724514, -0.011244077235460281, 0.01890231855213642, -0.007134647108614445, 0.04366467520594597, 0.0019330715294927359, -0.001348654506728053, -0.03587950021028519, 0.004318338818848133, 0.0014755867887288332, -0.013698099181056023, -0.005405195988714695, 0.014353915117681026, 0.01808783784508705, 0.00666922889649868, -0.039116270840168, -0.029025161638855934, 0.0273327324539423, -0.02068994753062725, -0.002837463514879346, -0.031986914575099945, -0.024476757273077965, 0.01512608677148819, 0.00987955555319786, 0.0035514570772647858, 0.02659229375422001, -0.038164280354

In [16]:
# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",     # 복합명사를 어떻게 색인을 할 것인가 정하는것. mixed 는 각각의 단어를 색인하고 합쳐진 단어도 색인함.
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]    # 여기서 지정한 것들을 색인하지 않음!
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {"type": "text", "analyzer": "nori"},
        "embedding": {
            "type": "dense_vector",
            #"dims": 768,
            "dims": len(embeddings[0]),
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [17]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [18]:
# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    #doc["embeddings"] = embedding.tolist()
    doc["embedding"] = embedding
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

(4272, [])


In [19]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [20]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 33.571957 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 18.898249 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 18.276913 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.


In [21]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.6708469 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.5462088 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 0.5266198 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.


## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [22]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- search api를 호출할 때, 사용자의 대화가 멀티턴이 아니라면, 사용자의 대화 내용을 그대로 standalone_query 에 대입한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [23]:
from sentence_transformers import CrossEncoder
cross_encoder_model = CrossEncoder("bongsoo/albert-small-kor-cross-encoder-v1")

/data/ephemeral/home/miniconda3/envs/colbert/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [24]:
# def merge_retrieve_result_from_elastic(standalone_query, result1, result2):
#     hits_merged_list = []
    
#     def append_without_duplicate(result):
#         for rst in result['hits']['hits']:
#             # 중복해서 추가되지 않도록.
#             is_found = False
            
#             for rst2 in hits_merged_list:
#                 if rst['_source']['docid'] == rst2['_source']['docid']:
#                     is_found = True
#                     break
            
#             if is_found: continue
            
#             hits_merged_list.append(rst)
    
#     # 중복 없이 hits_merged_list 에 추가
#     #append_without_duplicate(result1)
#     append_without_duplicate(result2)  
    
#     # 유사도 비교를 할 리스트 만들기
#     # pair_list = []    
#     # for rst in hits_merged_list:
#     #     pair_list.append( (standalone_query, rst['_source']['content']) )
    
#     # # 유사도 계산
#     # scores = cross_encoder_model.predict(pair_list)
    
#     # # 내림차순 정렬을 위해 유사도 리스트에 인덱스 추가
#     # scores_with_idx = []
#     # for i, score in enumerate(scores):
#     #     dic = {}
#     #     dic['idx'] = i
#     #     dic['score'] = score
#     #     scores_with_idx.append(dic)
    
#     # # score 기준으로 내림차순 정렬
#     # scores_with_idx = sorted(scores_with_idx, key=lambda x: x['score'], reverse=True)
    
#     # # 최종 결과 만들기
#     # ret = []
#     # for dic in scores_with_idx:
#     #     ret.append(hits_merged_list[dic['idx']])
    
#     # return ret
#     return hits_merged_list

In [25]:
def convert_elasticsearch_result_to_common_result(search_results):
    ret_list = []
    
    for rst in search_results['hits']['hits']:
        dic = {}
        
        dic['docid'] = rst['_source']['docid']
        dic['content'] = rst['_source']['content']
        dic['embedding'] = rst['_source']['embedding']
        
        ret_list.append(dic)
        
    return ret_list

In [26]:
def convert_colbert_result_to_common_result(search_results):
    pass

In [27]:
from collections import defaultdict
from typing import List

def reciprocal_rank_fusion(common_results_list:List[List[dict]], k=5):
    rrf = {}
    
    for common_results in common_results_list:
        for i, common_result in enumerate(common_results, 1):
            key = common_result['docid']
            
            if key not in rrf:
                dic = defaultdict(float)
                rrf[key] = dic
            else:
                dic = rrf[key]
            
            dic['rrf_score'] += 1.0 / (k + i)
            dic['content'] = common_result['content']
            dic['embedding'] = common_result['embedding']
    
    return sorted(rrf.items(), key=lambda x: x[1]['rrf_score'], reverse=True)

In [28]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")
        response["standalone_query"] = standalone_query

        results_sparse = sparse_retrieve(standalone_query, 3)
        results_dense = dense_retrieve(standalone_query, 3)
        
        common_results_1 = convert_elasticsearch_result_to_common_result(results_sparse)
        common_results_2 = convert_elasticsearch_result_to_common_result(results_dense)
        
        common_results_list = [common_results_1, common_results_2]
        rank_infos = reciprocal_rank_fusion(common_results_list)
        
        retrieved_context = []
        for i, rank_info in enumerate(rank_infos):
            if i >= 3:
                break   # 최대 3개까지만..
            
            docid = rank_info[0]
            content = rank_info[1]['content']
            score = rank_info[1]['rrf_score']
            
            retrieved_context.append(content)
            
            response["topk"].append(docid)
            response["references"].append({"score": score, "content": content})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [29]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 0:
            #     break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [30]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
submission_filename = "submission_024.csv"

eval_rag("../../data/eval.jsonl", submission_filename)

Test 0
Question: [{'role': 'user', 'content': 'Python 공부중이야.'}, {'role': 'assistant', 'content': '네 python은 배워 두시면 유용합니다.'}, {'role': 'user', 'content': 'list의 값중 가장 작은 값을 알려주는 함수가 뭐야?'}]
Answer: 가장 작은 값을 알려주는 함수는 `min()` 함수입니다. 리스트나 튜플과 같은 반복 가능한 객체의 요소 중에서 가장 작은 값을 반환합니다. 리스트 `l`이 주어졌을 때, `min(l)`은 리스트 `l`의 요소 중 가장 작은 값을 반환합니다.

Test 1
Question: [{'role': 'user', 'content': '평균을 구하는 코드를 작성중이야.'}, {'role': 'assistant', 'content': '네 무엇을 도와드릴까요?'}, {'role': 'user', 'content': '예외처리가 필요한 경우를 알려줘.'}]
Answer: 평균을 구하는 코드에서 예외처리가 필요한 경우는 다음과 같습니다:
1. 입력값이 없는 경우
2. 입력값이 숫자가 아닌 경우
3. 입력값이 음수인 경우
4. 입력값이 너무 큰 경우 (최대값을 초과하는 경우)

이러한 경우에 대비하여 코드를 작성하시면 좋을 것 같습니다. 추가적인 도움이 필요하시면 언제든지 말씀해주세요!

Test 2
Question: [{'role': 'user', 'content': 'python 공부중인데...'}, {'role': 'assistant', 'content': '네 꼭 필요한 언어라서 공부해 두면 좋습니다.'}, {'role': 'user', 'content': '숫자 계산을 위한 operator 우선순위에 대해 알려줘.'}]
Answer: 숫자 계산을 위한 operator 우선순위는 일반적으로 괄호 > 지수 > 곱셈/나눗셈 > 덧셈/뺄셈 순서로 적용됩니다. 이 순서에 따라 연산이 수행되며, 괄호 안의 연산이 가장 먼저 이루어지고